___
<a href='https://cafe.naver.com/jmhonglab'><p style="text-align:center;"><img src='https://lh3.googleusercontent.com/lY3ySXooSmwsq5r-mRi7uiypbo0Vez6pmNoQxMFhl9fmZJkRHu5lO2vo7se_0YOzgmDyJif9fi4_z0o3ZFdwd8NVSWG6Ea80uWaf3pOHpR4GHGDV7kaFeuHR3yAjIJjDgfXMxsvw=w2400'  class="center" width="50%" height="50%"/></p></a>
___
<center><em>Content Copyright by HongLab, Inc.</em></center>

### 총알(Bullet)

In [ ]:
import pygame

pygame.init()

screen = pygame.display.set_mode((1024, 768))  # 윈도우 크기
clock = pygame.time.Clock()

ground_y = 512 # 바닥 위치

# Bullet
img = pygame.image.load("present-gift-box-reward-full.png").convert_alpha()
w, h = img.get_size()
img = pygame.transform.scale(img, (w // 10, h // 10))
vx = 10.0 # 총알 속도
vy = 0.0
x = 240
y = 400
rect = img.get_rect()
rect.center = (x, y)

running = True

while running:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    """업데이트"""

    # x += vx*dt에서 dt 곱하기를 생략 (대신 vx를 작은 속도값 사용)
    # 예) vx가 100이고 dt가 0.01이라면 
    #     dt를 1로 가정해서 곱하기를 생략하고 vx를 1로 사용해도 같은 효과
    x += vx
    y += vy
    rect.center = (x, y)

    """화면에 그리기"""

    screen.fill((255, 255, 255))
    pygame.draw.rect(screen, color=(128, 128, 128), rect=pygame.Rect(0, 64*8, 1024, 64*4))

    screen.blit(img, rect)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()

##### [실습] 총알을 여러 개 만들기 위해서 클래스로 정리
- class Bullet 만들기
- bullet_group = [] 사용
- K_LCTRL 키를 누를 때마다 총알 추가
- 총알이 화면 오른쪽을 벗어나면 삭제

In [ ]:
# 실습용
import pygame
from pygame.locals import *

class Bullet:
    def __init__(self, x, y):
        pass

    def update(self):
        pass

    def draw(self, surface):
        pass

pygame.init()

screen = pygame.display.set_mode((1024, 768))  # 윈도우 크기
clock = pygame.time.Clock()

bullet_group = [] # 총알의 그룹

scorefont = pygame.font.SysFont("system", 48)
ground_y = 512 # 바닥 위치

running = True

while running:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            quit = True
            running = False

    """업데이트"""


    """화면에 그리기"""

    screen.fill((255, 255, 255))
    pygame.draw.rect(screen, color=(128, 128, 128), rect=pygame.Rect(0, 64*8, 1024, 64*4))

    for b in bullet_group:
        b.draw(screen)

    # 디버깅 용도
    scoretext = scorefont.render(f"Num bullets = {len(bullet_group)}", 1, (0, 0, 0))
    screen.blit(scoretext, (700, 10))

    pygame.display.flip()
    clock.tick(30)


pygame.quit()

##### 총알에 회전 추가

In [ ]:
# 위 코드에 이어서 직접 추가해보세요

### [선택] 총알 그룹 클래스 만들기

In [ ]:
import pygame
from pygame.locals import *

class Bullet:
    img_src = None

    def __init__(self, x, y):
        if Bullet.img_src == None:
            Bullet.img_src = pygame.image.load("present-gift-box-reward-full.png").convert_alpha()
            w, h = Bullet.img_src.get_size()
            Bullet.img_src = pygame.transform.scale(Bullet.img_src, (w // 10, h // 10))
        self.image = Bullet.img_src # 첫 프레임은 회전이 없음
        self.vx = 20.0 # 총알 속도
        self.vy = 0.0
        self.x = x
        self.y = y
        self.angle = 0.0
        self.rect = self.image.get_rect()
        self.rect.center = (self.x, self.y)

    def update(self):
        self.x += self.vx
        self.y += self.vy
        self.angle -= 20.0 # 각속도
        self.image = pygame.transform.rotate(Bullet.img_src, self.angle)
        self.rect = self.image.get_rect(center=(self.x, self.y))

    def draw(self, surface):
        surface.blit(self.image, self.rect)

class BulletGroup:
    def __init__(self):
        self.bullets = []

    def add(self, bullet):
        self.bullets.append(bullet)

    def update(self):
        for b in self.bullets.copy(): # copy 주의
            b.update()
            if b.rect.left > 1024:
                self.bullets.remove(b)

    def draw(self, *args): # 가변인수
        for b in self.bullets:
            b.draw(*args) # 가변인수

pygame.init()

screen = pygame.display.set_mode((1024, 768))  # 윈도우 크기
clock = pygame.time.Clock()

bullet_group = BulletGroup() # 그룹

ground_y = 512 # 바닥 위치

running = True

while running:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            quit = True
            running = False
        elif event.type == KEYUP and event.key == K_LCTRL:
            bullet_group.add(Bullet(240, 400))

    """업데이트"""
    bullet_group.update()

    """화면에 그리기"""

    screen.fill((255, 255, 255))
    pygame.draw.rect(screen, color=(128, 128, 128), rect=pygame.Rect(0, 64*8, 1024, 64*4))

    bullet_group.draw(screen)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()

### 총알 충돌처리

```object_rect.colliderect(bullet.rect)```

In [ ]:
import pygame
from pygame.locals import *

class Bullet:
    img_src = None

    def __init__(self, x, y):
        if Bullet.img_src == None:
            Bullet.img_src = pygame.image.load("present-gift-box-reward-full.png").convert_alpha()
            w, h = Bullet.img_src.get_size()
            Bullet.img_src = pygame.transform.scale(Bullet.img_src, (w // 10, h // 10))
        self.image = Bullet.img_src # 첫 프레임은 회전이 없음
        self.vx = 20.0 # 총알 속도
        self.vy = 0.0
        self.x = x
        self.y = y
        self.angle = 0.0
        self.rect = self.image.get_rect()
        self.rect.center = (self.x, self.y)

    def update(self):
        self.x += self.vx # dt 곱하기를 생략 (작은 속도값 사용)
        self.y += self.vy
        self.rect.center = (self.x, self.y)
        self.angle -= 20.0
        self.image = pygame.transform.rotate(Bullet.img_src, self.angle)
        self.rect = self.image.get_rect(center=self.rect.center)

    def draw(self, surface):
        surface.blit(self.image, self.rect)

        # 디버깅 용도
        pygame.draw.rect(screen, (255, 0, 0), self.rect, 1)

class BulletGroup:
    def __init__(self):
        self.bullets = []

    def add(self, bullet):
        self.bullets.append(bullet)

    def update(self):
        for b in self.bullets:
            b.update()

    def draw(self, *args):
        for b in self.bullets:
            b.draw(*args)

pygame.init()

screen = pygame.display.set_mode((1024, 768))  # 윈도우 크기
clock = pygame.time.Clock()

object_rect = pygame.Rect((900, 380, 100, 100))

bullet_group = BulletGroup() # 그룹

ground_y = 512 # 바닥 위치

running = True

while running:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            quit = True
            running = False
        elif event.type == KEYUP and event.key == K_LCTRL:
            bullet_group.add(Bullet(240, 400))

    """업데이트"""
    bullet_group.update()
    
    for b in bullet_group.bullets.copy(): # copy 주의
        if object_rect.colliderect(b.rect):
            bullet_group.bullets.remove(b)


    """화면에 그리기"""

    screen.fill((255, 255, 255))
    pygame.draw.rect(screen, color=(128, 128, 128), rect=pygame.Rect(0, 64*8, 1024, 64*4))
    pygame.draw.rect(screen, color=(0, 255, 0), rect=object_rect)
    bullet_group.draw(screen)
    pygame.display.flip()
    clock.tick(30)

pygame.quit()

### 음악 재생

```
# 배경 음악 추가
mixer.init()
mixer.music.load("jingle-bells-violin-loop-8645.mp3")
mixer.music.set_volume(0.1)
mixer.music.play()

# 사운드 추가
break_sound = mixer.Sound("422669__lynx-5969__ice-break-with-hand.wav")
break_sound.set_volume(0.3)
break_sound.play()
```

In [2]:
# 위 코드에 이어서 직접 추가해보세요

### [선택] 스프라이트 그룹 ([pygame.sprite.Group](https://www.pygame.org/docs/ref/sprite.html#pygame.sprite.Group)) 사용법

- pygame.sprite.Sprite 상속
- self.image와 self.rect 필요

In [ ]:
import pygame
from pygame.locals import *

class Bullet(pygame.sprite.Sprite):
    img_src = None

    def __init__(self, x, y):
        super().__init__()
        if Bullet.img_src == None:
            Bullet.img_src = pygame.image.load("present-gift-box-reward-full.png").convert_alpha()
            w, h = Bullet.img_src.get_size()
            Bullet.img_src = pygame.transform.scale(Bullet.img_src, (w // 10, h // 10))
        self.image = Bullet.img_src # 첫 프레임은 회전이 없음
        self.vx = 20.0 # 총알 속도
        self.vy = 0.0
        self.x = x
        self.y = y
        self.angle = 0.0
        self.rect = self.image.get_rect()
        self.rect.center = (self.x, self.y)

    def update(self):
        self.x += self.vx # dt 곱하기를 생략 (작은 속도값 사용)
        self.y += self.vy
        self.rect.center = (self.x, self.y)
        self.angle -= 20.0
        self.image = pygame.transform.rotate(Bullet.img_src, self.angle)
        self.rect = self.image.get_rect(center=self.rect.center)

    # 디버깅용
    def draw_rect(self, screen):
        pygame.draw.rect(screen, (255, 0, 0), self.rect, 2)

pygame.init()

screen = pygame.display.set_mode((1024, 768))  # 윈도우 크기
clock = pygame.time.Clock()

bullet_group = pygame.sprite.Group()

ground_y = 512 # 바닥 위치

running = True

while running:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            quit = True
            running = False
        elif event.type == KEYUP and event.key == K_LCTRL:
            bullet_group.add(Bullet(240, 400))

    """업데이트"""
    bullet_group.update()
    
    for b in bullet_group.copy(): # copy 주의
        if b.rect.left > 1024:
            bullet_group.remove(b)

    """화면에 그리기"""

    screen.fill((255, 255, 255))
    pygame.draw.rect(screen, color=(128, 128, 128), rect=pygame.Rect(0, 64*8, 1024, 64*4))

    bullet_group.draw(screen)

    for b in bullet_group:
        b.draw_rect(screen)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()

### 랜덤하게 장애물 생성

In [ ]:
import pygame
from pygame.locals import *
import random

class Obstacle(pygame.sprite.Sprite): # class Bullet과 비슷합니다.
    img_src = None

    def __init__(self, x, y, vx=0.0, vy=0.0, av=0.0, scale=1):
        super().__init__()
        if Obstacle.img_src == None:
            Obstacle.img_src = pygame.image.load("wintertileset/png/Object/IceBox.png").convert_alpha()
            w, h = Obstacle.img_src.get_size()
            Obstacle.img_src = pygame.transform.scale(Obstacle.img_src, (w // scale, h // scale))
        self.image = Obstacle.img_src # 첫 프레임은 회전이 없음
        self.x = x
        self.y = y
        self.vx = vx
        self.vy = vy
        self.av = av # 각 속도 (angular velocity)
        self.angle = 0.0
        self.rect = self.image.get_rect()
        self.rect.center = (self.x, self.y)

    def update(self):
        self.x += self.vx # dt 곱하기 생략
        self.y += self.vy
        self.rect.center = (self.x, self.y)
        self.angle -= self.av
        self.image = pygame.transform.rotate(Obstacle.img_src, self.angle)
        self.rect = self.image.get_rect(center=self.rect.center)


pygame.init()

screen = pygame.display.set_mode((1024, 768))  # 윈도우 크기
clock = pygame.time.Clock()

obstacles = pygame.sprite.Group()

ground_y = 512 # 바닥 위치

running = True

while running:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            quit = True
            running = False
        elif event.type == KEYUP and event.key == K_LCTRL:
            bullet_group.add(Bullet(240, 400))

    """업데이트"""
    obstacles.update()
    
    # 확률을 바꿔보세요.
    if random.random() > 0.98:  # 프레임당 2%의 확률로 장애물 생성
        obstacles.add(Obstacle(1000, 480, vx=-5))

    for o in obstacles.copy(): # copy 주의
        if o.rect.right < 0:
            obstacles.remove(o)

    """화면에 그리기"""

    screen.fill((255, 255, 255))
    pygame.draw.rect(screen, color=(128, 128, 128), rect=pygame.Rect(0, 64*8, 1024, 64*4))

    obstacles.draw(screen)

    pygame.display.flip()
    clock.tick(30)

pygame.quit()

### 기타

아이템
- 화면 오른쪽 바깥 쪽에서 랜덤하게 만들고 왼쪽으로 이동

폭발 효과
- 캐릭터와 비슷하게 스프라이트 애니메이션 사용
- 다만, 애니메이션을 반복하는 것이 아니라 폭발이 끝나면 삭제
- 스프라이트는 캔디와 같은 폴더에 explosion으로 시작하는 파일들

더 해볼 수 있는 것들
- 클래스와 모듈 만들어서 코드 정리
- 바닥이 없는 곳 만들어서 빠지면 사망
- 장애물에도 움직이는 스프라이트 애니메이션 사용
- 다른 스타일의 게임 만들기 (비행기 슈팅, 테트리스 등)

